# Формирование дополнительных признаков
В этом ноутбуке выполняется получение дополнительных признаков из текстов сообщений в соцсети. Это должно улучшить качество модели классификации постов на понравившиеся и не понравившиеся конкретному пользователю.

In [1]:
import pandas as pd
import numpy as np
from gensim.models import doc2vec


SAVE_DATA_TO_CSV = False

## Загрузка подготовленных данных
Перед загрузкой данных с признаками надо сначала выполнить все действия в ноутбуке `prepare_and_save_features.ipynb`, предварительно поставив флаг `SAVE_DATA_TO_CSV = True`. Это требуется для получения файла `./data/post_text.csv`. Если этот файл уже существует и содержит актуальные данные, то выполнять `prepare_and_save_features.ipynb` не нужно.

In [2]:
df_post_text = pd.read_csv("../data/post_text.csv")
print(df_post_text.shape)
df_post_text.head(3)

(7023, 3)


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business


## Подготовка тренировочных данных
Здесь выполняется преобразование списка текстов сообщений в датасет для использования при обучении модели doc2vec.

In [3]:
def get_prepared_text(in_text: str):
    """
     Функция предобработки текста.
     Выполняет удаление групп пробелов и переносы
     и приводит текст к нижнему регистру.
    """
    in_text = in_text.replace('\n', ' ')
    while '  ' in in_text:
        in_text = in_text.replace('  ', ' ')

    in_text = in_text.strip().lower()
    return in_text


def get_tagged_document_from_df(df):
    """
    Функция подготовки текстового датасета
    для использования в модели doc2vec.
    """
    for idx in df.index:
        prep_txt = get_prepared_text(df.loc[idx]["text"])
        yield doc2vec.TaggedDocument(prep_txt.split(), [idx])


data_train = list(get_tagged_document_from_df(df_post_text))

## Обучение модели

Создание модели и словаря для модели:

In [4]:
d2v_model = doc2vec.Doc2Vec(vector_size=20, min_count=2, epochs=200)

d2v_model.build_vocab(data_train)

Обучение модели:

In [5]:
%%time
d2v_model.train(data_train, 
                total_examples=d2v_model.corpus_count, 
                epochs=d2v_model.epochs)

CPU times: user 11min 39s, sys: 13.3 s, total: 11min 52s
Wall time: 4min 58s


## Получение признаков из векторов текстов
Здесь происходит получение векторного представления сообщений и на основе этого представления формирование дополнительных числовых признаков для каждого сообщения. 

In [6]:
%%time
texts_vectors = []
for row in data_train:
    row_vector = []
    row_vector.extend(row.tags)
    row_vector.extend(d2v_model.infer_vector(row.words))
    texts_vectors.append(row_vector)
len(texts_vectors)

CPU times: user 7min 32s, sys: 292 ms, total: 7min 33s
Wall time: 7min 33s


7023

In [7]:
d2v_array = np.array(texts_vectors)
idxs = d2v_array[:, 0].astype(int)
vals = d2v_array[:, 1:]

In [8]:
df_d2v_features = pd.DataFrame(data=vals, 
                               index=idxs, 
                               columns=[f"d2v_{i+1}" 
                                        for i in range(vals.shape[1])])
df_d2v_features.head(3)

,d2v_1,d2v_2,d2v_3,d2v_4,d2v_5,d2v_6,d2v_7,d2v_8,d2v_9,d2v_10,d2v_11,d2v_12,d2v_13,d2v_14,d2v_15,d2v_16,d2v_17,d2v_18,d2v_19,d2v_20
0,0.475043,9.604814,-1.963183,3.234909,-0.696753,-4.904148,2.217891,13.679256,-2.375175,-14.787580,2.283673,-1.398498,1.748283,4.383441,-3.488338,7.282822,-2.106024,-12.426719,-1.144862,1.471610
1,-0.479926,-1.825975,-3.626417,-0.409148,-4.444747,-2.822081,1.921235,5.102415,-4.266790,-7.294027,0.139248,-0.641879,-0.757251,-1.580353,0.675212,5.340289,-0.766597,-4.552031,0.140839,1.277483
2,-1.183734,9.598454,-0.787468,5.477929,-2.863148,1.828389,0.888420,5.811047,-6.261268,-15.021671,6.362102,-0.072358,3.763139,-1.317846,-3.896773,8.344177,-0.654131,-5.567890,-3.583399,2.895062


## Сохранение полученных признаков
Здесь полученные дополнительные признаки объединяются с исходным датасетом и сохраняются в файл для дальнейшего использования.

In [9]:
if SAVE_DATA_TO_CSV:
    df_post_data = pd.concat([df_post_text[["post_id"]], 
                              df_d2v_features], axis=1)
    df_post_data.to_csv("../data/post_features_d2v.csv", 
                        index=False)